# Work on Data from SQL

## Do the usefulls Imports

In [1]:
import json
import sqlite3 as sql
import pandas as pd
from datetime import datetime
from datetime import date
from functools import reduce

with open("Localisation.json","r") as my_file :
    dict_localisation = json.load(my_file)

## Prepare SQL Connexion and Data Size

In [2]:
connexion = sql.connect("./bce.db")
data_size = 500000

## Extract Table Code from Database and make it a DataFrame

In [3]:
df_code_sql = pd.read_sql_query("SELECT Category,Code,Description FROM code WHERE code.Language = 'FR'",connexion)
#df_code_sql

## Extract List of Category Unique Value and Create a Codes' Dictionary 

In [4]:
df_code = df_code_sql
list_Category = list(df_code["Category"].unique())
dict_codes = {category : dict(zip(df_code[df_code["Category"]==category]["Code"],df_code[df_code["Category"]==category]["Description"])) for category in list_Category}

## Extract Table Enterprise from Database and make it a DataFrame

In [5]:
df_enterprise_sql = pd.read_sql_query(f"SELECT EnterpriseNumber,JuridicalSituation,TypeOfEnterprise,JuridicalForm,StartDate FROM enterprise LIMIT {data_size}",connexion)
#df_enterprise_sql

## Adjust Enterprise DataFrame Content

In [6]:
df_enteprise = df_enterprise_sql
df_enteprise = df_enteprise.rename(columns={"EnterpriseNumber":"EntityNumber"})
df_enteprise["JuridicalSituation"] = df_enteprise["JuridicalSituation"].map(dict_codes["JuridicalSituation"])
df_enteprise["TypeOfEnterprise"] = df_enteprise["TypeOfEnterprise"].map(dict_codes["TypeOfEnterprise"])
df_enteprise["JuridicalForm"] = df_enteprise["JuridicalForm"].map(dict_codes["JuridicalForm"])
df_enteprise["StartDate"] = pd.to_datetime(df_enteprise["StartDate"])
df_enteprise["StartYear"] = df_enteprise["StartDate"].dt.year
df_enteprise["Age"] = date.today().year - df_enteprise["StartDate"].dt.year + date.today().month/12 - df_enteprise["StartDate"].dt.month/12 + date.today().day/365.25 - df_enteprise["StartDate"].dt.day/365.25

## Extract Table Denomination from Database and make it a DataFrame

In [7]:
df_denomination_sql = pd.read_sql_query(f"SELECT DISTINCT EntityNumber,Denomination FROM denomination WHERE TypeOfDenomination = '002' LIMIT {data_size}",connexion)
#df_denomination_sql

## Adjust Denomination DataFrame Content

In [8]:
df_denomination = df_denomination_sql
#df_denomination

## Extract Table Address from Database and make it a DataFrame

In [9]:
df_address_sql = pd.read_sql_query(f"SELECT DISTINCT EntityNumber,Zipcode FROM address LIMIT {data_size}",connexion)
#df_address_sql

## Adjust Table Address DataFrame Content

In [10]:
df_address = df_address_sql

for key in dict_localisation.keys() :

    df_address[key] = df_address["Zipcode"].map(dict_localisation[key])

#df_address

## Extract Table Activity from Database and make it a DataFrame

In [11]:
df_activity_sql = pd.read_sql_query(f"SELECT EntityNumber,ActivityGroup,NaceCode,Classification FROM activity LIMIT {3*data_size}",connexion)

## Adjust Activity DataFrame Content

In [12]:
df_activity = df_activity_sql[["EntityNumber","ActivityGroup","Classification","NaceCode"]]

def nace_code_section(row) :

    Nace_check = int(int(row["NaceCode"])/1000)

    if  Nace_check < 4 :
        return "A"
    elif Nace_check < 10 :
        return "B"
    elif Nace_check < 34 :
        return "C"
    elif Nace_check == 35 :
        return "D"
    elif Nace_check < 40 :
        return "E"
    elif Nace_check < 44 :
        return "F"
    elif Nace_check < 48 :
        return "G"
    elif Nace_check < 54 :
        return "H"
    elif Nace_check < 57 :
        return "I"
    elif Nace_check < 64 :
        return "J"
    elif Nace_check < 67 :
        return "K"
    elif Nace_check == 68 :
        return "L"
    elif Nace_check < 76 :
        return "M"
    elif Nace_check < 83 :
        return "N"
    elif Nace_check == 84 :
        return "O"
    elif Nace_check == 85 :
        return "P"
    elif Nace_check < 89 :
        return "Q"
    elif Nace_check < 94 :
        return "R"
    elif Nace_check < 97 :
        return "S"
    elif Nace_check < 99 :
        return "T"
    elif Nace_check == 99 :
        return "U"         

df_activity["ActivityGroup"]= df_activity["ActivityGroup"].map(dict_codes["ActivityGroup"])
df_activity["Classification"]= df_activity["Classification"].map(dict_codes["Classification"])
df_activity["NaceCodeSection"] = df_activity.apply(nace_code_section,axis=1)
df_activity["NaceDescription"] = df_activity["NaceCodeSection"].map(dict_codes["Nace2008"])
df_activity["NaceDescription"] = [str(value)[13:] for value in df_activity["NaceDescription"]]
#df_activity

## Merge DataFrames and Save as CSV file

In [13]:
list_dataframes = [df_activity,df_enteprise,df_denomination,df_address]
df_totaly_merged = reduce(lambda df_merged,next_df : pd.merge(df_merged,next_df, how = "inner", on = "EntityNumber"), list_dataframes)
df_totaly_merged.to_csv("bce.csv")
#df_totaly_merged